# best

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob  
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 17

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.2, mlp_dropout=0.2):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=30)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [6]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 0):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=3,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [7]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [8]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64

In [9]:

def feature_engineer(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)

       # 결측치
       df = df.replace(' ', np.nan)
       df = df.fillna(0)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       market_cols = ['단가(원)', '거래량'] # , '경매건수', '거래대금(원)'

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = ['해당일자_전체거래물량(kg)', '일자별_도매가격_평균(원)','하위가격 평균가(원)', '상위가격 평균가(원)' ] 
       # '하위가격 거래물량(kg)', '상위가격 거래물량(kg)', '일자별_도매가격_최대(원)', , '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)', 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['cos_time'] # ,, '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = ['주산지_초기온도','주산지_최대온도', '주산지_최저온도', '주산지_평균온도', '주산지_강수량'] 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['datadate', '해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열

       df = df[[c for c in df.columns if c in selected_columns]]

       return df

&nbsp;

## Train 과정

In [10]:
from sklearn.preprocessing import MinMaxScaler

losses = []
scalers = {}

for i in tqdm(data_list):
    print(i)
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    # 사용할 열 선택 및 index 설정
    df = feature_engineer(df)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.replace({' ': np.nan})
    df = df.fillna(0)
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.002, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

  0%|          | 0/37 [00:00<?, ?it/s]

./aT_data/data/train/train_4.csv
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-22 00:45:55.939207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-22 00:45:55.939648: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-22 00:45:56.767446: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-22 00:45:56.968028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 00:45:57.531552: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  3%|▎         | 1/37 [01:31<55:05, 91.83s/it]


valid_loss:  1006770.5625
./aT_data/data/train/train_15.csv


2022-09-22 00:47:27.535176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 00:47:27.732118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  5%|▌         | 2/37 [03:03<53:24, 91.56s/it]

...


valid_loss:  90160.4375
./aT_data/data/train/train_18.csv


2022-09-22 01:57:56.278836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 01:57:56.567919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
 97%|█████████▋| 36/37 [1:46:08<14:55, 895.73s/it]


valid_loss:  106173592.0
./aT_data/data/train/train_9.csv


2022-09-22 02:32:03.877651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:32:04.138626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 37/37 [2:02:56<00:00, 199.36s/it]


valid_loss:  201084.890625


In [11]:
losses

[1006770.5625,
 529000.6875,
 289105.21875,
 73780.0703125,
 23475406.0,
 4440080.0,
 5304746.5,
 2451664.75,
 1897049.0,
 71059784.0,
 1064347.25,
 1025612.9375,
 947423.5625,
 1849428.375,
 206623.59375,
 325607.625,
 4446025.0,
 3852172.5,
 4172862.5,
 744357.5,
 279906.90625,
 1067658.75,
 7181153.0,
 459736.25,
 158551.03125,
 2652659.0,
 2300019.5,
 7336653.5,
 9925762.0,
 3122627.0,
 774249.5,
 3944540.75,
 4613155.5,
 320961.75,
 90160.4375,
 106173592.0,
 201084.890625]

&nbsp;

## Test 과정

In [12]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)

        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택 및 index 설정

        df = feature_engineer(df)
        df = df[[c for c in df.columns if c != '해당일자_전체평균가격(원)']]

        df.set_index('datadate', drop=True, inplace=True)
        
        # # nan 처리
        # df = df.replace({' ': np.nan})
        # df = df.fillna(0)

        # # train input 과 형상 맞추기
        # add_col = [i for i in check_col if i not in df.columns]

        # for a in add_col:
        #     df[a] = 0

        # # ' ' -> nan 으로 변경
        # for a in df.columns:
        #     df[a] = df[a].replace({' ': np.nan})

        # # nan 처리
        # df = df.fillna(0)
        # # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]2022-09-22 02:48:52.448534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 32ms/step


2022-09-22 02:48:52.854053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:48:52.944269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:48:53.037633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 34ms/step


2022-09-22 02:48:53.137724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:48:53.227986: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:48:53.325306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-09-22 02:49:26.161336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-22 02:49:26.250816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 30ms/step


2022-09-22 02:49:26.335280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 29ms/step


2022-09-22 02:49:26.424569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 10/10 [00:34<00:00,  3.49s/it]


&nbsp;

## 정답 제출 파일생성

In [13]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  9.36it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [14]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.12500,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-1755.51540,-428.310600,-899.836100,-904.880430,-1071.717000,-1890.552500,185.665470,-1383.559200,-1668.901600,-283.745030,...,-3156.816700,-317.845860,-370.382080,-370.479980,-1410.476900,-2561.183600,-1862.741700,-1592.230100,-972.629700,-696.067200
2,-2169.26510,-548.203060,-1304.013300,-1207.942300,-1364.035600,-2279.715800,124.714400,-1968.657500,-2215.343500,-506.667140,...,-3871.513700,-436.600650,-619.164900,-464.467770,-1636.667000,-3445.241500,-2288.213000,-2024.710600,-1205.245800,-829.958100
3,-1995.95730,-572.635600,-1078.157200,-1145.684300,-1224.753200,-1978.088500,118.568160,-2026.429000,-1990.579500,-836.541560,...,-3894.800300,-378.427460,-591.457900,-757.406100,-1522.274300,-3141.059800,-2449.217800,-2096.052500,-1157.750400,-752.670600
4,-1940.91490,-548.355300,-650.151370,-1084.408400,-1089.647800,-1822.651100,56.793137,-2009.034300,-1833.287600,-370.153870,...,-3857.970500,-312.538120,-406.988250,-449.226350,-1521.986600,-2752.106000,-2180.901400,-2047.164400,-1154.472000,-475.630650
5,-1934.88920,-483.496830,-170.133160,-1098.354000,-1093.964100,-1994.030600,119.908775,-1959.534900,-1635.962300,-60.942760,...,-3794.941200,-295.715300,-296.688700,-82.421580,-1565.927500,-2861.763200,-1939.751300,-1919.257100,-1199.873300,-458.847080
6,-1571.24730,-365.120540,-74.432884,-869.724700,-909.200500,-1466.739300,60.985703,-1559.334200,-994.586360,-122.583855,...,-3217.728800,-229.878370,-156.712950,-82.678700,-1293.252000,-2216.881000,-1400.765700,-1516.632300,-961.667800,-355.787500
7,-284.64220,-36.517190,-110.142180,-180.288330,-245.407380,-232.561100,-4.779504,-627.051640,-338.430050,-70.123060,...,-768.979900,-76.955956,-83.501470,-73.971870,-155.542560,-356.412050,-178.997620,-325.240420,-271.127900,-108.157840
8,-1687.93090,-410.907780,-955.073240,-995.953500,-1201.967000,-1868.619500,41.500980,-1442.305200,-1768.748500,-334.810420,...,-3195.259500,-324.758330,-429.291400,-414.331760,-1300.497700,-2823.542500,-1891.184400,-1635.226900,-1110.917000,-718.867900
9,-2145.82130,-517.698850,-1322.154700,-1263.616300,-1464.479700,-2210.911600,76.311165,-1902.141500,-2212.106400,-535.494400,...,-3567.559800,-426.933960,-642.840450,-514.126300,-1448.006500,-3537.013700,-2282.080800,-1916.444500,-1282.793300,-805.319300


- 변동률 계산 

In [15]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값
    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)
    
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [16]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [17]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.453490,-0.314445,-0.309245,-0.266135,-0.271471,-0.204297,0.068328,-0.411647,-0.339767,-0.241893,...,-0.335062,-0.236536,-0.388191,-0.469536,-0.331497,-0.304406,-0.334801,-0.333417,0,d+1
1,-0.560371,-0.402464,-0.448148,-0.355269,-0.345517,-0.246351,0.045897,-0.585730,-0.451016,-0.431934,...,-0.560121,-0.296543,-0.450443,-0.631609,-0.407214,-0.387088,-0.414872,-0.397551,0,d+2
2,-0.515601,-0.420401,-0.370528,-0.336958,-0.310236,-0.213756,0.043635,-0.602919,-0.405257,-0.713153,...,-0.535056,-0.483572,-0.418959,-0.575844,-0.435867,-0.400727,-0.398523,-0.360531,0,d+3
3,-0.501383,-0.402576,-0.223436,-0.318937,-0.276013,-0.196959,0.020901,-0.597743,-0.373234,-0.315557,...,-0.368178,-0.286812,-0.418880,-0.504538,-0.388117,-0.391381,-0.397395,-0.227828,0,d+4
4,-0.499826,-0.354960,-0.058469,-0.323038,-0.277107,-0.215479,0.044128,-0.583016,-0.333061,-0.051954,...,-0.268396,-0.052623,-0.430974,-0.524641,-0.345201,-0.366927,-0.413023,-0.219789,0,d+5
5,-0.405889,-0.268054,-0.025580,-0.255796,-0.230305,-0.158499,0.022444,-0.463945,-0.202485,-0.104503,...,-0.141769,-0.052787,-0.355928,-0.406416,-0.249283,-0.289953,-0.331027,-0.170423,0,d+6
6,-0.073530,-0.026809,-0.037852,-0.053025,-0.062163,-0.025131,-0.001759,-0.186565,-0.068900,-0.059780,...,-0.075539,-0.047228,-0.042808,-0.065340,-0.031855,-0.062180,-0.093328,-0.051808,0,d+7
7,-0.436031,-0.301668,-0.328228,-0.292921,-0.304464,-0.201927,0.015273,-0.429126,-0.360095,-0.285426,...,-0.388354,-0.264533,-0.357922,-0.517634,-0.336558,-0.312626,-0.382402,-0.344339,0,d+8
8,-0.554315,-0.380069,-0.454382,-0.371644,-0.370960,-0.238916,0.028084,-0.565940,-0.450357,-0.456510,...,-0.581539,-0.328248,-0.398520,-0.648433,-0.406123,-0.366390,-0.441566,-0.385749,0,d+9
9,-0.497509,-0.397508,-0.382575,-0.355014,-0.335071,-0.209682,0.019428,-0.564848,-0.413117,-0.698727,...,-0.532722,-0.497562,-0.373984,-0.581233,-0.420690,-0.373311,-0.406895,-0.347797,0,d+10


- 정답 양식으로 변경

In [18]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [19]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.453490,-0.314445,-0.309245,-0.266135,-0.271471,-0.204297,0.068328,-0.411647,...,-0.365338,-0.527978,-0.335062,-0.236536,-0.388191,-0.469536,-0.331497,-0.304406,-0.334801,-0.333417
1,0,d+2,-0.560371,-0.402464,-0.448148,-0.355269,-0.345517,-0.246351,0.045897,-0.585730,...,-0.448050,-0.725243,-0.560121,-0.296543,-0.450443,-0.631609,-0.407214,-0.387088,-0.414872,-0.397551
2,0,d+3,-0.515601,-0.420401,-0.370528,-0.336958,-0.310236,-0.213756,0.043635,-0.602919,...,-0.450745,-0.628611,-0.535056,-0.483572,-0.418959,-0.575844,-0.435867,-0.400727,-0.398523,-0.360531
3,0,d+4,-0.501383,-0.402576,-0.223436,-0.318937,-0.276013,-0.196959,0.020901,-0.597743,...,-0.446482,-0.519161,-0.368178,-0.286812,-0.418880,-0.504538,-0.388117,-0.391381,-0.397395,-0.227828
4,0,d+5,-0.499826,-0.354960,-0.058469,-0.323038,-0.277107,-0.215479,0.044128,-0.583016,...,-0.439188,-0.491217,-0.268396,-0.052623,-0.430974,-0.524641,-0.345201,-0.366927,-0.413023,-0.219789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.067534,-0.212668,-0.171184,-0.132016,-0.222891,-0.053962,-0.002723,NaN,...,-0.043108,0.071165,-0.141434,-0.175684,-0.152009,-0.083392,-0.191056,-0.096892,-0.149172,-0.075420
146,9,d+12,-0.048000,-0.203423,-0.081414,-0.126789,-0.206864,-0.046236,-0.006859,NaN,...,-0.045542,0.146284,0.034059,0.021111,-0.153362,0.022379,-0.178557,-0.088449,-0.136405,-0.081287
147,9,d+13,-0.035347,-0.169401,-0.087759,-0.107162,-0.163667,-0.059107,-0.007241,NaN,...,-0.041185,0.116371,0.009425,-0.049868,-0.133194,-0.001207,-0.162878,-0.067319,-0.129592,-0.136730
148,9,d+14,-0.007142,-0.021297,0.006092,-0.028906,-0.043534,-0.002256,0.001624,NaN,...,-0.014627,0.001828,0.007582,-0.008175,0.000259,0.001457,0.016327,-0.009263,-0.029832,-0.008645


- 제출 양식

In [20]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [21]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.453490,-0.314445,-0.309245,-0.266135,-0.271471,-0.204297,0.068328,-0.411647,...,-0.365338,-0.527978,-0.335062,-0.236536,-0.388191,-0.469536,-0.331497,-0.304406,-0.334801,-0.333417
1,0,d+2,-0.560371,-0.402464,-0.448148,-0.355269,-0.345517,-0.246351,NaN,-0.585730,...,-0.448050,-0.725243,-0.560121,-0.296543,-0.450443,-0.631609,-0.407214,-0.387088,-0.414872,-0.397551
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.043635,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.501383,-0.402576,-0.223436,-0.318937,-0.276013,-0.196959,NaN,-0.597743,...,-0.446482,-0.519161,-0.368178,-0.286812,-0.418880,-0.504538,-0.388117,-0.391381,-0.397395,-0.227828
4,0,d+5,-0.499826,-0.354960,-0.058469,-0.323038,-0.277107,-0.215479,NaN,-0.583016,...,-0.439188,-0.491217,-0.268396,-0.052623,-0.430974,-0.524641,-0.345201,-0.366927,-0.413023,-0.219789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.067534,-0.212668,-0.171184,-0.132016,-0.222891,-0.053962,-0.002723,NaN,...,-0.043108,0.071165,-0.141434,-0.175684,-0.152009,-0.083392,-0.191056,-0.096892,-0.149172,-0.075420
146,9,d+12,-0.048000,-0.203423,-0.081414,-0.126789,-0.206864,-0.046236,-0.006859,NaN,...,-0.045542,0.146284,0.034059,0.021111,-0.153362,0.022379,-0.178557,-0.088449,-0.136405,-0.081287
147,9,d+13,-0.035347,-0.169401,-0.087759,-0.107162,-0.163667,-0.059107,-0.007241,NaN,...,-0.041185,0.116371,0.009425,-0.049868,-0.133194,-0.001207,-0.162878,-0.067319,-0.129592,-0.136730
148,9,d+14,-0.007142,-0.021297,0.006092,-0.028906,-0.043534,-0.002256,0.001624,NaN,...,-0.014627,0.001828,0.007582,-0.008175,0.000259,0.001457,0.016327,-0.009263,-0.029832,-0.008645
